In [1]:
import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

junction_exons = pd.read_csv('{}/junction_exons.csv'.format(folder), index_col=0)
print junction_exons.shape

(706732, 3)


/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:784: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


## Build graph database

In [ ]:
from poshsplice.junctions_to_events import JunctionAggregator
%time ja = JunctionAggregator(junction_exons, junction_col='junction_location')

CPU times: user 2min 32s, sys: 644 ms, total: 2min 33s
Wall time: 2min 33s


## SE Events

In [28]:
exons_to_junctions = pd.read_csv('{}/exons_to_junctions_se.csv'.format(folder), header=None)
exons_to_junctions = exons_to_junctions.rename(columns={0: 'exon1', 1: 'exon2', 2: 'exon3', 3:'junctions'})
exons_to_junctions['junctions'] = exons_to_junctions['junctions'].map(eval)
junctions = exons_to_junctions['junctions'].apply(pd.Series).rename(
    columns={0:'junction_exon12', 1:'junction_exon23', 2: 'junction_exon13'})
exons_to_junctions = pd.concat([exons_to_junctions, junctions], axis=1)
exons_to_junctions['event_id'] = exons_to_junctions['exon1'] + '@' \
    + exons_to_junctions['exon2'] + '@' + \
    exons_to_junctions['exon3']
exons_to_junctions.head()

,exon1,exon2,exon3,junctions,junction_exon12,junction_exon23,junction_exon13,event_id
0,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100188908-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",chr10:100189647-100190327:-,chr10:100189400-100189547:-,chr10:100189400-100190327:-,exon:chr10:100190328-100190427:-@exon:chr10:10...
1,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100188913-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",chr10:100189647-100190327:-,chr10:100189400-100189547:-,chr10:100189400-100190327:-,exon:chr10:100190328-100190427:-@exon:chr10:10...
2,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-,"(chr10:100189647-100190327:-, chr10:100189400-...",chr10:100189647-100190327:-,chr10:100189400-100189547:-,chr10:100189400-100190327:-,exon:chr10:100190328-100190427:-@exon:chr10:10...
3,exon:chr10:100193697-100193848:-,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-,"(chr10:100191049-100193696:-, chr10:100190428-...",chr10:100191049-100193696:-,chr10:100190428-100190887:-,chr10:100190428-100193696:-,exon:chr10:100193697-100193848:-@exon:chr10:10...
4,exon:chr10:100195392-100195529:-,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-,"(chr10:100195172-100195391:-, chr10:100193849-...",chr10:100195172-100195391:-,chr10:100193849-100195028:-,chr10:100193849-100195391:-,exon:chr10:100195392-100195529:-@exon:chr10:10...


In [30]:
se_df = exons_to_junctions.drop('junctions', axis=1)
se_df.head()

,exon1,exon2,exon3,junction_exon12,junction_exon23,junction_exon13,event_id
0,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100188908-100189399:-,chr10:100189647-100190327:-,chr10:100189400-100189547:-,chr10:100189400-100190327:-,exon:chr10:100190328-100190427:-@exon:chr10:10...
1,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100188913-100189399:-,chr10:100189647-100190327:-,chr10:100189400-100189547:-,chr10:100189400-100190327:-,exon:chr10:100190328-100190427:-@exon:chr10:10...
2,exon:chr10:100190328-100190427:-,exon:chr10:100189548-100189646:-,exon:chr10:100189330-100189399:-,chr10:100189647-100190327:-,chr10:100189400-100189547:-,chr10:100189400-100190327:-,exon:chr10:100190328-100190427:-@exon:chr10:10...
3,exon:chr10:100193697-100193848:-,exon:chr10:100190888-100191048:-,exon:chr10:100190328-100190427:-,chr10:100191049-100193696:-,chr10:100190428-100190887:-,chr10:100190428-100193696:-,exon:chr10:100193697-100193848:-@exon:chr10:10...
4,exon:chr10:100195392-100195529:-,exon:chr10:100195029-100195171:-,exon:chr10:100193697-100193848:-,chr10:100195172-100195391:-,chr10:100193849-100195028:-,chr10:100193849-100195391:-,exon:chr10:100195392-100195529:-@exon:chr10:10...


In [31]:
se_df.to_csv('{}/exons_to_junctions_se.csv'.format(folder))

## MXE events

In [ ]:
%time mxe_events = ja.mutually_exclusive_exon()

In [18]:
pd.options.display.max_colwidth = 50

In [23]:
mxe_df = pd.Series(mxe_events).reset_index()
mxe_df = mxe_df.rename(columns=dict(('level_{}'.format(i), 'exon{}'.format(i+1)) for i in range(4)))
mxe_df = mxe_df.rename(columns={0: 'junctions'})
mxe_df['junction13'] = mxe_df.junctions.map(lambda x: x[0])
mxe_df['junction34'] = mxe_df.junctions.map(lambda x: x[1])
mxe_df['junction12'] = mxe_df.junctions.map(lambda x: x[2])
mxe_df['junction24'] = mxe_df.junctions.map(lambda x: x[3])
mxe_df['event_id'] = mxe_df.exon1 + '@' + mxe_df.exon2 + '@' \
    + mxe_df.exon3 + '@' + mxe_df.exon4
mxe_df.head()

,exon1,exon2,exon3,exon4,junctions,junction13,junction34,junction12,junction24,event_id
0,exon:chr10:102003455-102003529:-,exon:chr10:101997779-101997988:-,exon:chr10:101996607-101996726:-,exon:chr10:101995424-101995521:-,"[chr10:101996727-102003454:-, chr10:101995522-...",chr10:101996727-102003454:-,chr10:101995522-101996606:-,chr10:101997989-102003454:-,chr10:101995522-101997778:-,exon:chr10:102003455-102003529:-@exon:chr10:10...
1,exon:chr10:102003455-102003534:-,exon:chr10:101997779-101997988:-,exon:chr10:101996607-101996726:-,exon:chr10:101995424-101995521:-,"[chr10:101996727-102003454:-, chr10:101995522-...",chr10:101996727-102003454:-,chr10:101995522-101996606:-,chr10:101997989-102003454:-,chr10:101995522-101997778:-,exon:chr10:102003455-102003534:-@exon:chr10:10...
2,exon:chr10:102013178-102013296:-,exon:chr10:102010005-102010089:-,exon:chr10:102003455-102003534:-,exon:chr10:101997779-101997988:-,"[chr10:102003535-102013177:-, chr10:101997989-...",chr10:102003535-102013177:-,chr10:101997989-102003454:-,chr10:102010090-102013177:-,chr10:101997989-102010004:-,exon:chr10:102013178-102013296:-@exon:chr10:10...
3,exon:chr10:102013178-102014376:-,exon:chr10:102010005-102010089:-,exon:chr10:102003455-102003534:-,exon:chr10:101997779-101997988:-,"[chr10:102003535-102013177:-, chr10:101997989-...",chr10:102003535-102013177:-,chr10:101997989-102003454:-,chr10:102010090-102013177:-,chr10:101997989-102010004:-,exon:chr10:102013178-102014376:-@exon:chr10:10...
4,exon:chr10:102020723-102020785:-,exon:chr10:102019975-102020076:-,exon:chr10:102016019-102016233:-,exon:chr10:102013178-102013296:-,"[chr10:102016234-102020722:-, chr10:102013297-...",chr10:102016234-102020722:-,chr10:102013297-102016018:-,chr10:102020077-102020722:-,chr10:102013297-102019974:-,exon:chr10:102020723-102020785:-@exon:chr10:10...


In [25]:
mxe_df.shape

(17748, 9)

In [24]:
mxe_df = mxe_df.drop('junctions', axis=1)
mxe_df.head()

,exon1,exon2,exon3,exon4,junction13,junction34,junction12,junction24,event_id
0,exon:chr10:102003455-102003529:-,exon:chr10:101997779-101997988:-,exon:chr10:101996607-101996726:-,exon:chr10:101995424-101995521:-,chr10:101996727-102003454:-,chr10:101995522-101996606:-,chr10:101997989-102003454:-,chr10:101995522-101997778:-,exon:chr10:102003455-102003529:-@exon:chr10:10...
1,exon:chr10:102003455-102003534:-,exon:chr10:101997779-101997988:-,exon:chr10:101996607-101996726:-,exon:chr10:101995424-101995521:-,chr10:101996727-102003454:-,chr10:101995522-101996606:-,chr10:101997989-102003454:-,chr10:101995522-101997778:-,exon:chr10:102003455-102003534:-@exon:chr10:10...
2,exon:chr10:102013178-102013296:-,exon:chr10:102010005-102010089:-,exon:chr10:102003455-102003534:-,exon:chr10:101997779-101997988:-,chr10:102003535-102013177:-,chr10:101997989-102003454:-,chr10:102010090-102013177:-,chr10:101997989-102010004:-,exon:chr10:102013178-102013296:-@exon:chr10:10...
3,exon:chr10:102013178-102014376:-,exon:chr10:102010005-102010089:-,exon:chr10:102003455-102003534:-,exon:chr10:101997779-101997988:-,chr10:102003535-102013177:-,chr10:101997989-102003454:-,chr10:102010090-102013177:-,chr10:101997989-102010004:-,exon:chr10:102013178-102014376:-@exon:chr10:10...
4,exon:chr10:102020723-102020785:-,exon:chr10:102019975-102020076:-,exon:chr10:102016019-102016233:-,exon:chr10:102013178-102013296:-,chr10:102016234-102020722:-,chr10:102013297-102016018:-,chr10:102020077-102020722:-,chr10:102013297-102019974:-,exon:chr10:102020723-102020785:-@exon:chr10:10...


In [26]:
mxe_df.to_csv('{}/exons_to_junctions_mxe.csv'.format(folder))

In [ ]:
# %time se_events = ja.skipped_exon()

In [ ]:
se_events

In [ ]:
mxe_events

In [ ]:
range(3)